In [1]:
import numpy as np
import cv2
from sklearn.neural_network import MLPRegressor
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib as mpl
from sklearn.model_selection import train_test_split

Reading output signal

In [2]:
# read video frame
def dep_cam_reduced(dep_v_name):
    dep_v = cv2.VideoCapture(dep_v_name)
    ret, frame = dep_v.read()
    counter=0

    frame_count = int(dep_v.get(cv2.CAP_PROP_FRAME_COUNT))

#     frame_height = int(frame.shape[0]/5)
#     frame_width = int(frame.shape[1]/5)
#     frame_height = frame.shape[0]
#     frame_width = frame.shape[1]
    frame_height = 30
    frame_width = 40
    depth_frames = np.empty((frame_count, frame_height, frame_width))

    while(dep_v.isOpened()):
        ret, frame = dep_v.read()
        if ret == False:
            break
        if ret == True:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_frame = cv2.resize(gray_frame, (frame_width, frame_height), interpolation = cv2.INTER_AREA)
            depth_frames[counter] = gray_frame
            counter+=1

    dep_v.release()
    cv2.destroyAllWindows()
    return depth_frames

### Read input depth data

In [3]:
# read output depth images data with reduced dimension
#left 32 as fresh unseen data
foldername = r'Test_Subject_Leo/'
test_folder = ['test24', 'test30', 'test31','test33', 'test35' ]
test_num = ['24', '30', '31', '33', '35']
sub_name = foldername + test_folder[0]
dep_name = sub_name + r'/depth_processed_leo_test' + test_num[0] + '.avi';
depth =  dep_cam_reduced(dep_name)/255.0
for i in range(1, 5):
    subfolder_name = foldername + test_folder[i]
    dep_video_name = subfolder_name + r'/depth_processed_leo_test' + test_num[i] + '.avi'
    #print(dep_video_name) 
    dep_v_temp =  dep_cam_reduced(dep_video_name)/255.0
    depth = np.concatenate((depth, dep_v_temp))

### Read output data

In [4]:
foldername = r'Test_Subject_Leo/'
test_folder = ['test24', 'test30', 'test31','test33', 'test35' ]
test_num = ['24', '30', '31', '33', '35']
subfolder_name = foldername + test_folder[0]
qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[0] + '_' + '11_15_2020' + '.txt'
theta_z = pd.read_csv(qtm_file)
theta_z = theta_z.iloc[:,0].values
for i in range(1,5):
    subfolder_name = foldername + test_folder[i]
    if test_folder[i] == 'test30' or test_folder[i] == 'test31':
        qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[i] + '_' + '11_24_2020' + '.txt'
    else: 
        qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[i] + '_' + '11_25_2020' + '.txt'
    dataset_y =  pd.read_csv(qtm_file)
    theta_z_temp = dataset_y.iloc[:,0].values
    theta_z = np.concatenate((theta_z, theta_z_temp))

### Discretization of output data (theta z)

In [5]:
def saturate(theta, min_val, max_val):
    for i in range(len(theta)):
        if theta[i] < min_val:
            theta[i] = min_val
            continue
        if theta[i] > max_val:
            theta[i] = max_val
            continue
    return theta           

In [6]:
# Creates label based on number of classes. Given theta_z, the goal is to put each angle to a certain class or bin. 
# each class will have min_

def create_label(n_class, theta_z, min_val, max_val):
    

    labels = []
    class_list = []

    # interval of each class
    interval = (max_val - min_val) / n_class 
    print(interval)

    # create a list containing min and max bound of each class. This way we don't have to have a lot of if statements
    for j in range(n_class):
        min_class = min_val + (j) * interval # min_class is the minimum bound of each class

        max_class = min_class + interval # max_class is the maximum bound of each class

        class_list.append([min_class, max_class])


    # Go through each theta and assign a label. The final labels will have same number of elements as 
    for i in range(len(theta_z)):
        for j in range (len(class_list)):
            class_range = class_list[j]    
            if class_range[0] <= theta_z[i] < class_range[1]:
                labels.append(j)
                break
                
       
    return(labels)

In [7]:
# try different number of classes
n_class = 5
min_val = -50 
max_val = 50

sat_theta_z = saturate(theta_z, min_val, max_val)
labeled_z = create_label(n_class, theta_z, min_val, max_val)

# plt.plot(sat_theta_z)
# plt.plot(labeled_z,'--')
# plt.legend(['Ground Truth', 'Discretized'])
# plt.xlabel('Sample', fontsize=18)
# plt.ylabel('Angle (degree)', fontsize=18)

20.0


### Read force data

In [ ]:
print(len(labeled_z))
print(sat_theta_z.shape)

In [8]:
subfolder_name = foldername + test_folder[0]
force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[0] + '_' + '11_15_2020' + '.txt'
force = pd.read_csv(force_file)
force = force.iloc[:,:].values
for i in range(1,5):
    subfolder_name = foldername + test_folder[i]
    if test_folder[i] == 'test30' or test_folder[i] == 'test31':
        force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[i] + '_' + '11_24_2020' + '.txt'
    else: 
        force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[i] + '_' + '11_25_2020' + '.txt'
    dataset_y =  pd.read_csv(force_file)
    force_temp = dataset_y.iloc[:,:].values
    force = np.concatenate((force, force_temp))

In [9]:
# Normalize data with Leo's BMI
weight = 67
height = 1.74
BMI = weight/(height**2)
force = force/BMI

### Split the data

In [10]:
depth = depth[0:len(labeled_z)]

force = force[0:len(labeled_z)]
labeled_z = np.array(labeled_z)
depth_train, depth_test, force_train, force_test, theta_z_train, theta_z_test = train_test_split(depth, force, labeled_z, test_size = 0.2, random_state = 0)

### Create neural network

In [11]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
tf.keras.backend.clear_session()

In [12]:
input_depth = layers.Input(shape = (30, 40))
input_force = layers.Input(shape = (6, ))

# first branch
x = layers.Flatten(input_shape=(30, 40))(input_depth)
x = layers.Dense(units=200, activation='relu')(x)
x = layers.Dense(units=200, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(100, activation='relu')(x)
x = tf.keras.Model(inputs= input_depth, outputs = x)

# second branch
y = layers.Dense(units=200, activation='relu')(input_force)
y = layers.Dense(units=200, activation='relu')(y)
y = layers.Dropout(0.2)(y)
y = layers.Dense(100, activation='relu')(y)
y = tf.keras.Model(inputs= input_force, outputs = y)

# combine two branches
combined = layers.concatenate([x.output, y.output])

# regression to one output
z = layers.Dense(units=100, activation='relu')(combined)
z = layers.Dropout(0.2)(z)
z = layers.Dense(units=100, activation='relu')(combined)
z = layers.Dense(units = 5, activation = 'linear')(z)

model_fd = tf.keras.Model(inputs = [x.input, y.input], outputs = z)

model_fd.compile(optimizer='adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                 metrics=['accuracy'])

history = model_fd.fit([depth_train, force_train],theta_z_train, verbose = 1, epochs = 300, batch_size = 128,
                   validation_data = ([depth_test, force_test], theta_z_test))

Epoch 1/300
438/438 [==============================] - 1s 3ms/step - loss: 0.8976 - accuracy: 0.6369 - val_loss: 0.8144 - val_accuracy: 0.6560
Epoch 2/300
438/438 [==============================] - 1s 2ms/step - loss: 0.8121 - accuracy: 0.6520 - val_loss: 0.7726 - val_accuracy: 0.6669
Epoch 3/300
438/438 [==============================] - 1s 2ms/step - loss: 0.7659 - accuracy: 0.6615 - val_loss: 0.7903 - val_accuracy: 0.6727
Epoch 4/300
438/438 [==============================] - 1s 2ms/step - loss: 0.7255 - accuracy: 0.6781 - val_loss: 0.7011 - val_accuracy: 0.6937
Epoch 5/300
438/438 [==============================] - 1s 2ms/step - loss: 0.6932 - accuracy: 0.6904 - val_loss: 0.6740 - val_accuracy: 0.6993
Epoch 6/300
438/438 [==============================] - 1s 3ms/step - loss: 0.6623 - accuracy: 0.7064 - val_loss: 0.6508 - val_accuracy: 0.7100
Epoch 7/300
438/438 [==============================] - 1s 3ms/step - loss: 0.6362 - accuracy: 0.7179 - val_loss: 0.6448 - val_accuracy: 0.7142

Epoch 58/300
438/438 [==============================] - 1s 2ms/step - loss: 0.2270 - accuracy: 0.9072 - val_loss: 0.3130 - val_accuracy: 0.8786
Epoch 59/300
438/438 [==============================] - 1s 2ms/step - loss: 0.2276 - accuracy: 0.9062 - val_loss: 0.3301 - val_accuracy: 0.8698
Epoch 60/300
438/438 [==============================] - 1s 2ms/step - loss: 0.2294 - accuracy: 0.9063 - val_loss: 0.3062 - val_accuracy: 0.8793
Epoch 61/300
438/438 [==============================] - 1s 2ms/step - loss: 0.2179 - accuracy: 0.9105 - val_loss: 0.3423 - val_accuracy: 0.8678
Epoch 62/300
438/438 [==============================] - 1s 2ms/step - loss: 0.2220 - accuracy: 0.9094 - val_loss: 0.3209 - val_accuracy: 0.8784
Epoch 63/300
438/438 [==============================] - 1s 3ms/step - loss: 0.2178 - accuracy: 0.9100 - val_loss: 0.3166 - val_accuracy: 0.8804
Epoch 64/300
438/438 [==============================] - 1s 3ms/step - loss: 0.2180 - accuracy: 0.9108 - val_loss: 0.3250 - val_accuracy:

Epoch 115/300
438/438 [==============================] - 1s 2ms/step - loss: 0.1512 - accuracy: 0.9409 - val_loss: 0.3112 - val_accuracy: 0.8989
Epoch 116/300
438/438 [==============================] - 1s 2ms/step - loss: 0.1535 - accuracy: 0.9397 - val_loss: 0.3225 - val_accuracy: 0.8942
Epoch 117/300
438/438 [==============================] - 1s 2ms/step - loss: 0.1512 - accuracy: 0.9393 - val_loss: 0.3293 - val_accuracy: 0.8896
Epoch 118/300
438/438 [==============================] - 1s 2ms/step - loss: 0.1478 - accuracy: 0.9413 - val_loss: 0.3193 - val_accuracy: 0.8976
Epoch 119/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1456 - accuracy: 0.9418 - val_loss: 0.3236 - val_accuracy: 0.8901
Epoch 120/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1495 - accuracy: 0.9419 - val_loss: 0.3102 - val_accuracy: 0.8989
Epoch 121/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1481 - accuracy: 0.9403 - val_loss: 0.3263 - val_ac

438/438 [==============================] - 1s 3ms/step - loss: 0.1215 - accuracy: 0.9534 - val_loss: 0.3740 - val_accuracy: 0.8913
Epoch 172/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1201 - accuracy: 0.9529 - val_loss: 0.3294 - val_accuracy: 0.9061
Epoch 173/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1143 - accuracy: 0.9552 - val_loss: 0.3324 - val_accuracy: 0.9020
Epoch 174/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1146 - accuracy: 0.9560 - val_loss: 0.3381 - val_accuracy: 0.9006
Epoch 175/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1137 - accuracy: 0.9566 - val_loss: 0.3197 - val_accuracy: 0.9045
Epoch 176/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1154 - accuracy: 0.9555 - val_loss: 0.3153 - val_accuracy: 0.9105
Epoch 177/300
438/438 [==============================] - 1s 3ms/step - loss: 0.1170 - accuracy: 0.9546 - val_loss: 0.3498 - val_accuracy: 0.9017

438/438 [==============================] - 1s 2ms/step - loss: 0.0960 - accuracy: 0.9630 - val_loss: 0.3509 - val_accuracy: 0.9054
Epoch 228/300
438/438 [==============================] - 1s 2ms/step - loss: 0.1012 - accuracy: 0.9614 - val_loss: 0.3510 - val_accuracy: 0.9042
Epoch 229/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0982 - accuracy: 0.9630 - val_loss: 0.3286 - val_accuracy: 0.9126
Epoch 230/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0933 - accuracy: 0.9639 - val_loss: 0.3384 - val_accuracy: 0.9075
Epoch 231/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0992 - accuracy: 0.9621 - val_loss: 0.3326 - val_accuracy: 0.9087
Epoch 232/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0994 - accuracy: 0.9619 - val_loss: 0.3530 - val_accuracy: 0.9043
Epoch 233/300
438/438 [==============================] - 1s 2ms/step - loss: 0.0989 - accuracy: 0.9624 - val_loss: 0.3416 - val_accuracy: 0.9062

438/438 [==============================] - 1s 2ms/step - loss: 0.0856 - accuracy: 0.9687 - val_loss: 0.3578 - val_accuracy: 0.9075
Epoch 284/300
438/438 [==============================] - 1s 2ms/step - loss: 0.0841 - accuracy: 0.9681 - val_loss: 0.3698 - val_accuracy: 0.9067
Epoch 285/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0897 - accuracy: 0.9663 - val_loss: 0.3507 - val_accuracy: 0.9080
Epoch 286/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0867 - accuracy: 0.9677 - val_loss: 0.3772 - val_accuracy: 0.9077
Epoch 287/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0823 - accuracy: 0.9689 - val_loss: 0.3619 - val_accuracy: 0.9079
Epoch 288/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0859 - accuracy: 0.9683 - val_loss: 0.3379 - val_accuracy: 0.9104
Epoch 289/300
438/438 [==============================] - 1s 3ms/step - loss: 0.0838 - accuracy: 0.9689 - val_loss: 0.3500 - val_accuracy: 0.9087

In [13]:
# probability_model = tf.keras.Sequential([model_dp, tf.keras.layers.Softmax()])
predictions = model_fd.predict([depth_test, force_test])
y_predictions = np.argmax(predictions, axis = 1)
print(y_predictions.shape)

(13991,)


In [14]:
from sklearn import metrics
print(metrics.accuracy_score(theta_z_test, y_predictions))


0.9032949753412909


In [15]:
model_fd.save('my_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model/assets


In [16]:
from keras.models import load_model
model = load_model('my_model')
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
# import matplotlib as mpl
# mpl.rcParams['figure.figsize'] = (20,7)
# mpl.rc('xtick', labelsize=18) 
# mpl.rc('ytick', labelsize=18)
# plt.plot(theta_z_test[0:100])
# plt.plot(y_predictions[0:100],'--')
# plt.legend(['Ground Truth', 'Prediction'])
# plt.xlabel('Sample', fontsize=18)
# plt.ylabel('Angle (degree)', fontsize=18)
# plt.show()
# #plt.savefig('ground_v_prediction_zoomed in.png', dpi = 300)

In [17]:
foldername = r'Test_Subject_Leo/'
sub_name = foldername + 'test32'
dep_name = sub_name + r'/depth_processed_leo_test' + '32' + '.avi';
depth_fresh =  dep_cam_reduced(dep_name)/255.0


qtm_file = sub_name + r'/qtm_processed_leo_test' + '32' + '_' + '11_25_2020' + '.txt'
theta_z = pd.read_csv(qtm_file)
theta_z_fresh = theta_z.iloc[:,0].values
n_class = 5
min_val = -50 
max_val = 50

sat_theta_z = saturate(theta_z_fresh, min_val, max_val)
labeled_z_fresh = create_label(n_class, sat_theta_z, min_val, max_val)

force_file = sub_name + r'/fcss_processed_leo_test' + '32' + '_' + '11_25_2020' + '.txt'
force_fresh = pd.read_csv(force_file)
force_fresh = force_fresh/BMI

20.0


In [18]:
depth_fresh = depth_fresh[0:len(labeled_z_fresh)]

force_fresh = force_fresh[0:len(labeled_z_fresh)]

y_predic_unseen = model_fd.predict([depth_fresh, force_fresh])
y_predic_unseen = np.argmax(y_predic_unseen, axis = 1)

In [19]:
metrics.accuracy_score(labeled_z_fresh, y_predic_unseen)

0.5469126568603009

In [20]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(labeled_z_fresh, y_predic_unseen)

0.5338652040222721

In [ ]:
# mean_absolute_error(theta_z_test, y_prediction)
# mpl.rcParams['figure.figsize'] = (10,7)
# mpl.rc('xtick', labelsize=18) 
# mpl.rc('ytick', labelsize=18)
# plt.plot(labeled_z_fresh)
# plt.plot(y_predic_unseen,'--')
# plt.legend(['Ground Truth', 'Prediction'])
# plt.xlabel('Sample', fontsize=18)
# plt.ylabel('Angle (degree)', fontsize=18)
# plt.show()
# #plt.savefig('ground_v_prediction_zoomed in.png', dpi = 300)